In [ ]:

import pandas as pd
# Load the file as a Pandas DataFrame
data = pd.read_excel(r"C:\Users\Babak\Desktop\Pathogenomics\R\Pathogenomics_dataset_1_Copy.xlsx")



|**Descriptive Statistics of the dataset**|

In [ ]:
import numpy as np

num_vars = data.select_dtypes(include=[np.number]).columns
cat_vars = data.select_dtypes(include=['object']).columns


# Create an empty table
table = pd.DataFrame(columns=['Variable', 'Mean±SD', 'Count (%)'])

# Iterate over each numerical variable and get the mean and standard deviation
num_cols = data.select_dtypes(include=['float', 'int']).columns
for col in num_cols:
    mean = data[col].mean()
    std = data[col].std()
    table = table.append({'Variable': col, 'Mean ± SD': f"{mean:.2f} ± {std:.2f}", 'Count (%)': ''}, ignore_index=True)

# Iterate over each categorical variable and get the count and percentage of each category
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    categories = data[col].unique().tolist()
    if len(categories) == 1:
        count = data[col].count()
        percent = count / len(data) * 100
        table = table.append({'Variable': col, 'Mean ± SD': '', 'Count (%)': f"{count} ({percent:.2f}%)"},
                             ignore_index=True)
    else:
        for category in categories:
            count = data[data[col] == category][col].count()
            percent = count / len(data) * 100
            table = table.append({'Variable': f"{col} - {category}", 'Mean ± SD': '',
                                  'Count (%)': f"{count} ({percent:.2f}%)"},
                                 ignore_index=True)

# Save the table to an Excel file
table.to_excel('table.xlsx', index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts


fig, ax = plt.subplots(figsize=(6, 4))
kmfh = KaplanMeierFitter()
kmfh.fit(data['overall_survival'], event_observed=data['vital_status'])
kmfh.plot(ax=ax, show_censors =True)
plt.ylim(0, 1.01)
plt.xlabel("Time")
plt.ylabel("Probalities")
plt.legend(loc="best")
add_at_risk_counts(kmfh, ax=ax)
plt.show()

|**Main analyses**|

In [ ]:
data_processed = data.copy()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer


# separate the numerical and categorical variables
num_vars = data_processed.select_dtypes(include=[np.number]).columns
cat_vars = data_processed.select_dtypes(include=['object']).columns

# impute missing values for numerical variables with median
num_imputer = SimpleImputer(strategy='median')
data_processed[num_vars] = num_imputer.fit_transform(data_processed[num_vars])

# impute missing values for categorical variables with most frequent
cat_imputer = SimpleImputer(strategy='most_frequent')
data_processed[cat_vars] = cat_imputer.fit_transform(data_processed[cat_vars])

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# Convert string categorical variables to numeric labels
categorical_cols = ["ajcc_pathologic_stage", "ajcc_clinical_stage", "prior_malignancy", "prior_treatment", "ajcc_pathologic_t", "morphology", "ajcc_clinical_m", "ajcc_pathologic_n", "ajcc_clinical_n", "ajcc_clinical_t", "ajcc_pathologic_m", "alcohol_history", "race", "gender", "ethnicity", "paper_Methylation"]
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    data_processed[col] = label_encoders[col].fit_transform(data_processed[col])

# Convert integer columns to float64
data_processed = data_processed.astype({col: 'float64' for col in data_processed.select_dtypes(include='int32').columns})

# Standardize the continuous variables
continuous_cols = data_processed.columns.difference(categorical_cols + ['vital_status'] + ['overall_survival'])
scaler = StandardScaler()
data_processed[continuous_cols] = scaler.fit_transform(data_processed[continuous_cols])



# Verify that the data has been preprocessed correctly
print(data_processed.head())

In [ ]:
data_processed.dtypes

In [ ]:
non_numeric_cols = data_processed.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
print(non_numeric_cols)

In [ ]:
data_processed.isnull().sum()

In [ ]:
X = data_processed.drop(["overall_survival", "vital_status"], axis=1)
y = data_processed[['vital_status','overall_survival']]
y['vital_status'] = y['vital_status'].astype(int)

In [ ]:
print(X.shape)

In [ ]:
print(X.dtypes)


In [ ]:
non_numeric_columns = []
for column in X.columns:
    if X[column].dtype != 'float64' and X[column].dtype != 'int64':
        non_numeric_columns.append(column)
        
if len(non_numeric_columns) > 0:
    print("Non-numeric columns found: ", non_numeric_columns)
else:
    print("All columns are numeric")

In [ ]:
print(X)

In [ ]:
print(y['vital_status'])

In [ ]:
print(y)

In [ ]:
for col in data_processed.columns:
    if data_processed[col].isnull().any():
        print(f"{col} has missing values")


In [ ]:
import pandas as pd
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.svm import SVR
from sksurv.util import Surv
from sklearn.model_selection import KFold, cross_val_score
from sksurv.ensemble import RandomSurvivalForest
from lifelines.utils import survival_events_from_table
from sksurv.svm import FastSurvivalSVM
from sklearn.metrics import make_scorer
from sksurv.metrics import concordance_index_censored
from functools import partial, wraps
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
import tensorflow as tf
from keras import backend as K
from sksurv.metrics import (concordance_index_censored, concordance_index_ipcw,
                            brier_score, cumulative_dynamic_auc)
from sksurv.metrics import brier_score, integrated_brier_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt

np.seterr(over='ignore')

# Define loss function for deep learning model
def coxph_loss(y_true, y_pred):
    # extract the risk scores and event indicators
    rs = y_pred[:, 0]
    ei = y_true[:, 1]
    # calculate the log partial likelihood
    hazard_ratio = K.exp(rs)
    log_risk = K.log(K.cumsum(hazard_ratio))
    uncensored_likelihood = rs - log_risk
    censored_likelihood = uncensored_likelihood * ei
    neg_likelihood = -K.sum(censored_likelihood, axis=-1)
    return neg_likelihood

# Define function to create deep survival model
def create_deep_survival_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_shape=(input_dim,), activation="relu"))
    model.add(Dense(2, activation="softmax"))
    model.compile(optimizer="adam", loss=coxph_loss)
    return model

# Define models
models = [
    ("RSF", RandomSurvivalForest(n_estimators=100, max_depth=5, random_state=42)),
    ("GBSA", GradientBoostingSurvivalAnalysis(n_estimators=100, max_depth=5, random_state=42)),
    ("FastSVM", FastSurvivalSVM(alpha=1.0, max_iter=1000, random_state=42)),
    ("CoxPH", CoxPHSurvivalAnalysis(alpha=0.5, verbose=False)),
    ("DeepSurv", create_deep_survival_model)
]

# Define k-fold cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

def survival_events_from_table(df, event_col, time_col):
    """
    Convert a DataFrame to the event and time format expected by DeepSurv.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with columns for event indicator and time.
    event_col : str
        Name of the event column.
    time_col : str
        Name of the time column.

    Returns
    -------
    tuple
        Tuple of (event_indicator, observed_time) arrays.

    """
    df = df.set_index(time_col)
    event_indicator = df[event_col].values.astype(bool)
    observed_time = df.index.values.astype(float)
    return event_indicator, observed_time


# Run k-fold cross-validation
for model_name, model in models:
    print(model_name)
    c_indexes = []
    #X_important was derived from the feature importance analyses i.e. top 10 predictors see code below. run code below before this otherwise this does not work and should be X.iloc and kf.split(X).so X_important should be replaced with X if not using the top 10 predictors but the whole feature set
    for train_index, test_index in kf.split(X_important):
        X_train, y_train = X_important.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X_important.iloc[test_index], y.iloc[test_index]

        if model_name == "DeepSurv":
            # transform y_train using StandardScaler
            scaler = StandardScaler()
            y_train_censored = survival_events_from_table(y_train, 'vital_status', 'overall_survival')
            y_train_transformed_1 = scaler.fit_transform(y_train_censored[0].reshape(-1, 1)).flatten()
            y_train_transformed_2 = y_train_censored[1].reshape(-1, 1)
            y_train_transformed = np.concatenate([y_train_transformed_1.reshape(-1, 1), y_train_transformed_2], axis=1)
            y_train_transformed = pd.DataFrame(y_train_transformed, columns=['T', 'E'])
            # transform y_test using the same scaler
            y_test_censored = survival_events_from_table(y_test, 'vital_status', 'overall_survival')
            y_test_transformed_1 = scaler.transform(y_test_censored[0].reshape(-1, 1)).flatten()
            y_test_transformed_2 = y_test_censored[1].reshape(-1, 1)
            y_test_transformed = np.concatenate([y_test_transformed_1.reshape(-1, 1), y_test_transformed_2], axis=1)
            y_test_transformed = pd.DataFrame(y_test_transformed, columns=['T', 'E'])
            model_instance = KerasRegressor(build_fn=create_deep_survival_model, input_dim=X_train.shape[1], epochs=50, verbose=0) # instantiate the model
            model_instance.fit(X_train, y_train_transformed)  # fit the model
            y_pred = model_instance.predict(X_test)
            y_pred_risk = np.exp(-np.dot(y_pred, model_instance.model.layers[-1].get_weights()[0].T) - model_instance.model.layers[-1].get_weights()[1][0])
            # Compute predicted risk
            y_pred_log_odds = np.dot(y_pred, model_instance.model.layers[-1].get_weights()[0].T) + model_instance.model.layers[-1].get_weights()[1][0]
            y_pred_risk = 1 / (1 + np.exp(-y_pred_log_odds))
            y_pred_risk = np.mean(y_pred_risk, axis=1)
            c_index = concordance_index_censored(y_test['vital_status'].astype(bool), y_test['overall_survival'], np.ravel(y_pred[:, 0]))[0]
            c_indexes.append(c_index)
            print(f"Fold c-index: {c_index:.3f}")
            print(f"Mean c-index: {np.mean(c_indexes):.3f}\n")
            # Set threshold on predicted risk score
            threshold = 0.5
            y_pred_binary = np.zeros_like(y_pred_risk)
            y_pred_binary[y_pred_risk > threshold] = 1

            # Calculate ROC curve and AUC
            fpr, tpr, thresholds = roc_curve(y_test['vital_status'], y_pred_risk)
            roc_auc = auc(fpr, tpr)

            # Plot ROC curve
            plt.figure(figsize=(8, 8))
            plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic')
            plt.legend(loc="lower right")
            plt.show()

            print('\n')

            # Calculate AUC at specific time points
            y_pred_df = pd.DataFrame(y_pred_risk, index=y_test.index, columns=['survival_prob'])
            pool1year = y_test['overall_survival'] <= 365
            pool2year = y_test['overall_survival'] <= 730
            y_true_1year = y_test.loc[pool1year, 'vital_status']
            y_pred_1year = y_pred_df.loc[pool1year, 'survival_prob']
            fpr_1year, tpr_1year, thresholds_1year = roc_curve(y_true_1year, y_pred_1year)
            roc_auc_1year = auc(fpr_1year, tpr_1year)
            auc_list_1year_deepsurv = []
            auc_list_1year_deepsurv.append(roc_auc_1year)
            print(f"AUC after 1 year: {roc_auc_1year:.3f}")
            plt.plot(fpr_1year, tpr_1year, label=f"ROC Curve after 1 year (AUC = {roc_auc_1year:.3f})")

            y_true_2year = y_test.loc[pool2year, 'vital_status']
            y_pred_2year = y_pred_df.loc[pool2year, 'survival_prob']
            fpr_2year, tpr_2year, thresholds_2year = roc_curve(y_true_2year, y_pred_2year)
            roc_auc_2year = auc(fpr_2year, tpr_2year)
            auc_list_2year_deepsurv = []
            auc_list_2year_deepsurv.append(roc_auc_2year)

            print(f"AUC after 2 years: {roc_auc_2year:.3f}")
            plt.plot(fpr_2year, tpr_2year, label=f"ROC Curve after 2 years (AUC = {roc_auc_2year:.3f})")

            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curves - {model_name}')
            plt.legend()
            plt.show()
            # calculate the average AUC and the standard deviation over all folds
            avg_auc1deepsurv = np.mean(auc_list_1year_deepsurv)
            std_auc1deepsurv = np.std(auc_list_1year_deepsurv)
            avg_auc2deepsurv = np.mean(auc_list_2year_deepsurv)
            std_auc2deepsurv = np.std(auc_list_2year_deepsurv)
            # print the results
            print(f"Average AUC 1 year over {n_splits} folds: {avg_auc1deepsurv:.3f} +/- {std_auc1deepsurv:.3f}")
            print(f"Average AUC 2 year over {n_splits} folds: {avg_auc2deepsurv:.3f} +/- {std_auc2deepsurv:.3f}")


        else:
            y_train_struct = np.zeros(y_train.shape[0], dtype={'names':['vital_status', 'overall_survival'], 'formats':['?','f8']})
            y_train_struct['overall_survival'] = y_train['overall_survival']
            y_train_struct['vital_status'] = y_train['vital_status']
            model_instance = model
            model_instance.fit(X_train, y_train_struct)
            y_pred = model_instance.predict(X_test)
            y_test = y_test.copy()
            y_test.loc[:, 'vital_status'] = y_test['vital_status'].astype(bool)
            c_index = concordance_index_censored(y_test['vital_status'], y_test['overall_survival'], y_pred)[0]
            c_indexes.append(c_index)
            print(f"Fold c-index: {c_index:.3f}")
            print(f"Mean c-index: {np.mean(c_indexes):.3f}\n")
            threshold = np.median(y_test['overall_survival'])
            y_true = (y_test['overall_survival'] > threshold).astype(int)
            fpr, tpr, thresholds = roc_curve(y_true, y_pred)
            roc_auc = auc(fpr, tpr)
            print(roc_auc)
            plt.plot(fpr, tpr)
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curve - {model_name}')
            plt.show()
            
            print('\n')
            # Calculate AUC at specific time points
            y_pred_df = pd.DataFrame(y_pred, index=y_test.index, columns=['survival_prob'])
            pool1year = y_test['overall_survival'] <= 365
            pool2year = y_test['overall_survival'] <= 730
            y_true_1year = y_test.loc[pool1year, 'vital_status']
            y_pred_1year = y_pred_df.loc[pool1year, 'survival_prob']
            fpr_1year, tpr_1year, thresholds_1year = roc_curve(y_true_1year, y_pred_1year)
            roc_auc_1year = auc(fpr_1year, tpr_1year)
            auc_list_1year = []
            auc_list_1year.append(roc_auc_1year)
            print(f"AUC after 1 year: {roc_auc_1year:.3f}")
            plt.plot(fpr_1year, tpr_1year, label=f"ROC Curve after 1 year (AUC = {roc_auc_1year:.3f})")

            y_true_2year = y_test.loc[pool2year, 'vital_status']
            y_pred_2year = y_pred_df.loc[pool2year, 'survival_prob']
            fpr_2year, tpr_2year, thresholds_2year = roc_curve(y_true_2year, y_pred_2year)
            roc_auc_2year = auc(fpr_2year, tpr_2year)
            auc_list_2year = []
            auc_list_2year.append(roc_auc_2year)

            print(f"AUC after 2 years: {roc_auc_2year:.3f}")
            plt.plot(fpr_2year, tpr_2year, label=f"ROC Curve after 2 years (AUC = {roc_auc_2year:.3f})")

            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curves - {model_name}')
            plt.legend()
            plt.show()
            # calculate the average AUC and the standard deviation over all folds
            avg_auc1 = np.mean(auc_list_1year)
            std_auc1 = np.std(auc_list_1year)
            avg_auc2 = np.mean(auc_list_2year)
            std_auc2 = np.std(auc_list_2year)
            # print the results
            print(f"Average 1 year AUC over {n_splits} folds: {avg_auc1:.3f} +/- {std_auc1:.3f}")
            print(f"Average 2 year AUC over {n_splits} folds: {avg_auc2:.3f} +/- {std_auc2:.3f}")

|**Loss for deepcurve**|

In [ ]:
history = model_instance.fit(X_train, y_train_transformed, validation_data=(X_test, y_test_transformed), epochs=50, verbose=0)

# Get the train and validation loss values from DeepSurv
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the train and validation loss curves
import matplotlib.pyplot as plt

plt.plot(train_loss, label='train_loss')
plt.plot(val_loss, label='val_loss')
plt.legend()
plt.show()

|**ROC for deepcurve**|

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.calibration import calibration_curve

# Set threshold on predicted risk score
threshold = 0.1
y_pred_binary = np.zeros_like(y_pred_risk)
y_pred_binary[y_pred_risk > threshold] = 1

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test['vital_status'], y_pred_risk)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Calculate calibration curve
prob_true, prob_pred = calibration_curve(y_test['vital_status'], y_pred_risk, n_bins=10)

# Plot calibration curve
plt.figure(figsize=(8, 8))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.plot(prob_pred, prob_true, "s-", label="Model")
plt.xlabel("Predicted probability")
plt.ylabel("True probability")
plt.legend()
plt.title('Calibration curve')
plt.show()

|**Feature Importance**|

In [ ]:
from sklearn.inspection import permutation_importance
for model_name, model in models:
    print(model_name)
    if model_name == "DeepSurv":
        # Calculate feature importances
        print(type(model))
        result = permutation_importance(model_instance, X_train, y_train_transformed, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Print the feature importances
        for i in importance_indices:
            print(f"{X_train.columns[i]}: {importance_scores[i]}")
    else:
        # Calculate feature importances
        result = permutation_importance(model, X_train, y_train_struct, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Print the feature importances
        for i in importance_indices:
            print(f"{X_train.columns[i]}: {importance_scores[i]}")

|**Heatmap for feature importance seperately**|

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for model_name, model in models:
    print(model_name)
    if model_name == "DeepSurv":
        # Calculate feature importances
        result = permutation_importance(model_instance, X_train, y_train_transformed, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]


        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Create a heatmap of feature importances
        sns.set(rc={"figure.figsize":(15,10)})
        sns.set(font_scale=1.2)
        ax = sns.heatmap(importance_scores_pct.reshape(1, -1), annot=False, cmap="YlOrRd", fmt=".1f", cbar=False, xticklabels=False, yticklabels=False)
        #ax = sns.heatmap(importance_scores_pct.reshape(1, -1), annot=True, cmap="YlOrRd", fmt=".1f", cbar=False, xticklabels=X_train.columns[importance_indices], yticklabels=False)
        ax.set_title(f"Feature importance for {model_name}")
        plt.show()
        
    else:
        # Calculate feature importances
        result = permutation_importance(model, X_train, y_train_struct, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]

        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Create a heatmap of feature importances
        sns.set(rc={"figure.figsize":(15,10)})
        sns.set(font_scale=1.2)
        ax = sns.heatmap(importance_scores_pct.reshape(1, -1), annot=False, cmap="YlOrRd", fmt=".1f", cbar=False, xticklabels=False, yticklabels=False)
        #ax = sns.heatmap(importance_scores_pct.reshape(1, -1), annot=False, cmap="YlOrRd", fmt=".1f", cbar=False, xticklabels=X_train.columns[importance_indices], yticklabels=False)
        ax.set_title(f"Feature importance for {model_name}")
        
        plt.show()

|**Heatmap for feature importance pooled**|

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



# Create a dictionary to store the importance scores for each model
importance_scores_dict = {}

for model_name, model in models:
    
    print(model_name)
    if model_name == "DeepSurv":
        # Calculate feature importances
        result = permutation_importance(model_instance, X_train, y_train_transformed, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]

        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Min-max scaling of importance scores between 0 and 1
        importance_scores_norm = (importance_scores_pct - np.min(importance_scores_pct)) / (np.max(importance_scores_pct) - np.min(importance_scores_pct))
        
        # Add importance scores to the importance matrices dictionary
        importance_scores_dict[model_name] = importance_scores_norm


    else:
        # Calculate feature importances
        result = permutation_importance(model, X_train, y_train_struct, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1]

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]

        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Min-max scaling of importance scores between 0 and 1
        importance_scores_norm = (importance_scores_pct - np.min(importance_scores_pct)) / (np.max(importance_scores_pct) - np.min(importance_scores_pct))

        # Add the importance scores to the dictionary
        importance_scores_dict[model_name] = importance_scores_norm

# Concatenate the importance scores for all models
all_importance_scores = np.concatenate(list(importance_scores_dict.values()))

# Get the feature names
feature_names = X_train.columns.values

# Create a heatmap of feature importances for all models
sns.set(rc={"figure.figsize":(15,10)})
sns.set(font_scale=1.2)
ax = sns.heatmap(all_importance_scores.reshape(len(models), -1), annot=False, cmap="YlOrRd", fmt=".1f", cbar=False, xticklabels=False, yticklabels=[model[0] for model in models])
ax.set_title("Pooled feature importance")
plt.show()

|**Heatmap for feature importance pooled best 50 features**|

In [ ]:
sns.set(rc={"figure.figsize":(15,10)})
sns.set(font_scale=1.2)

for model_name, model in models:
    print(model_name)
    if model_name == "DeepSurv":
        # Calculate feature importances
        result = permutation_importance(model_instance, X_train, y_train_transformed, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1][:50] # Slice the first 50 indices

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]


        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Create a heatmap of feature importances
        ax = sns.heatmap(importance_scores_pct[importance_indices].reshape(1, -1), annot=True, cmap="Blues", fmt=".1f", cbar=False, xticklabels=X_train.columns[importance_indices], yticklabels=False)
        ax.set_title(f"Feature importance for {model_name}")
        plt.show()
        
    else:
        # Calculate feature importances
        result = permutation_importance(model, X_train, y_train_struct, n_repeats=10, random_state=0)

        # Get feature importance scores and their indices
        importance_scores = result.importances_mean
        importance_indices = np.argsort(importance_scores)[::-1][:50] # Slice the first 50 indices

        # Calculate the baseline C-index
        baseline = concordance_index_censored(y_test['vital_status'].astype(bool), y_pred_df['survival_prob'], y_test['overall_survival'])[0]

        # Calculate the percentage reduction in C-index for each feature
        importance_scores_pct = 100 * (baseline - result.importances_mean) / baseline

        # Create a heatmap of feature importances
        ax = sns.heatmap(importance_scores_pct[importance_indices].reshape(1, -1), annot=False, cmap="Blues", fmt=".1f", cbar=False, xticklabels=X_train.columns[importance_indices], yticklabels=False)
        ax.set_title(f"Feature importance for {model_name}")
        plt.xticks(rotation=90)

        plt.show()

|**plot feature importance**|

In [ ]:
import matplotlib.pyplot as plt

# Get feature importance scores and their indices
importance_scores = result.importances_mean
indices = np.argsort(importance_scores)[::-1]

# Plot feature importances
plt.bar(range(X_train.shape[1]), importance_scores[indices])
plt.xticks(range(X_train.shape[1]), X_train.columns[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.ylabel('Feature Importance')
plt.title('Feature Importances')
plt.show()

|**plot feature importance 50 best**|

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sort feature importances
indices = np.argsort(importance_scores)[::-1]

# Set number of features to display
n_features = 50

# Plot top N features
plt.figure(figsize=(10,10))
plt.barh(range(n_features), importance_scores[indices[:n_features]])
plt.yticks(range(n_features), X.columns[indices[:n_features]])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top {} Feature Importances'.format(n_features))
plt.show()

|**create new X DataFrame with top ten features to use for new improved prediction modelling**|

In [ ]:
# Get feature importance scores and their indices
importance_scores = result.importances_mean
importance_indices = np.argsort(importance_scores)[::-1][:10] # Get the top 10 important features

In [ ]:
X_important = pd.DataFrame()
# Add the important features to the dataframe
for i in importance_indices:
    feature_name = X_train.columns[i]
    X_important[feature_name] = X_train[feature_name]

In [ ]:
X_important